In [26]:
library("noisyr")
library("DESeq2")

library("EnhancedVolcano")
library(edgeR)
library(stringr)
library(gdata)
library(edgeR)
library(msigdbr)
library(hypeR)

In [5]:
new_library_paths<-c('/usr/local/lib/R/site-library')
.libPaths(new_library_paths)

In [6]:
.libPaths()

[1] "/usr/local/lib/R/site-library" "/usr/lib/R/site-library"      
[3] "/usr/lib/R/library"

In [ ]:
new_library_path <- "/my/new/library/path"
old_library_paths <- .libPaths()
new_library_paths <- c(old_library_paths, new_library_path)
.libPaths(new_library_paths)

In [9]:
#if (!require("BiocManager", quietly = TRUE))
install.packages("BiocManager")
BiocManager::install(version = "3.17", ask = FALSE)

BiocManager::install("edgeR",version=3.17)

Installing package into '/usr/local/lib/R/site-library'
(as 'lib' is unspecified)

Warning message in install.packages("BiocManager"):
"'lib = "/usr/local/lib/R/site-library"' is not writable"


ERROR: Error in install.packages("BiocManager"): unable to install packages


In [5]:
install.packages('msigdbr')

Installing package into '/data1/home/jaeminjj/R/x86_64-pc-linux-gnu-library/4.3'
(as 'lib' is unspecified)

also installing the dependency 'babelgene'




In [ ]:
if (!file.exists("~/.Renviron")) {
  file.create("~/.Renviron")
}

# open .Renviron file
file.edit("~/.Renviron")
R_LIBS_USER="new_library_dir"

[1] TRUE

# DEG + VOLCANO

In [27]:
data_types <- rep("COVID_BULK", 5)
data_typs <- list.files("/data1/projects/2021_MOPA/SMALL_NMF_V2/data/COVID_paper/")
data_typs <- data_typs[!data_typs == ".ipynb_checkpoints"]

data_typs1 <- paste0("COVID_paper/", data_typs)
a <- data_typs1
data_types1 <- rep("COVID_", 5)
data_types3 <- rep("_paper", 5)
b <- paste0(data_types1, data_typs, data_types3)
data_typs <- a
data_types <- b

path <- "/data1/projects/2021_MOPA/SMALL_NMF_V2/result/"
sub_sample <- "full"
method <- "NMF"
data_types1 <- c("COAD_paper", "STAD_paper", "BRCA_paper", "LUAD_paper", "COAD_filter_paper", "BRCA_filter_paper")
data_typs1 <- c("TCGA/COAD", "TCGA/STAD", "TCGA/BRCA", "TCGA/LUAD", "TCGA/COAD_filter", "TCGA/BRCA_filter")
data_types <- c(data_types1, data_types)
data_typs <- c(data_typs1, data_typs)

# DEG_BULK

In [29]:

sample_num<-0
sub_sample <-'full'
sample_num_len <-1
sub='yes'

sub='no'
subornot ='full_sample' ## sub_sample
numnum=0
#print(1)
mapply(function(data_type, data_typ) {
    methods<-list.files(path=paste0('/data/projects/2021_MOPA/SMALL_NMF_V2/result/',data_type,'/',sub_sample_num,'/remain_genes/'))
    #methods<-c('NMF_norm_10_360_0.001','noisyR','full_genes','PCA_norm_10_360_0.001')
    for(method in methods)
    {
        full_data <-read.table(paste0('/data/projects/2021_MOPA/SMALL_NMF_V2/data/',data_typ,'/full_data/gene_matrix.csv'),sep=',',header=1)
        sample_label<- read.table(paste0('/data/projects/2021_MOPA/SMALL_NMF_V2/data/',data_typ,'/full_data/subtype.csv'),sep=',',header=1)
                sample_label<- read.table(paste0('/data/projects/2021_MOPA/SMALL_NMF_V2/data/',data_typ,'/full_data/subtype.csv'),sep=',',header=1)
        NMF <- read.table(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/remain_genes/",method,"/",sample_num,".csv"), sep = ",",header=T)
        ####original data DESEQ2
        col_data<-sample_label
        rownames(col_data) <-col_data$X
            
        ######################################################### noisyR removal
        full<-full_data[full_data[,1] %in% NMF[,1],]
        rownames(full)<-full_data[full_data[,1]%in% NMF[,1],][,1]
        full<-full[,-c(1)]
        
        
        ## sub sample
        if(sub=='yes')
        {
            full<-full[,sub_sample_index[,2]+1]
            col_data<-col_data[sub_sample_index[,2]+1,]
        }
        ############### MAKE label combination
        len <-length(unique(sample_label$Subtype))
        #len<-len*(len-1)/2
        empty_list <- vector(mode = "list", length = (len*(len-1))/2)
        unique(sample_label$Subtype)[1]
        num<-1
        for( i in c(1:(length(unique(sample_label$Subtype))-1)))
        {    
            for(j in c((i+1):length(unique(sample_label$Subtype))))
                {
                    empty_list[[num]]<-append(empty_list[[num]],unique(sample_label$Subtype)[i])
                    empty_list[[num]]<-append(empty_list[[num]],unique(sample_label$Subtype)[j])
                    num<-num+1
            }
         }
        len<-len*(len-1)/2
        colnames(full)<-col_data[,1]
        
        
        for( i in c(1:len))
        {
            dds<-list()
            
            #col_data$Subtype<-as.factor(col_data$Subtype)
            ##?????
            rownames(col_data)<-col_data[,1]
            a<- DESeqDataSetFromMatrix(countData =full[,colnames(full)%in%rownames(col_data[col_data$Subtype %in% empty_list[[i]],])], 
                                           colData = col_data[col_data$Subtype %in% empty_list[[i]],],
                                           # correcting for effect of day (first value) + DEG between celltype (last value)
                                           design = ~ Subtype)
            print(i)
            dds <- estimateSizeFactors(a)
            dds<- DESeq(dds)
            res<-list()
            res[[1]]<-results(dds)
            res<-data.frame(res[[1]])
            res$rownames<-rownames(res)
            res<-as.data.frame(res)
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/DEG/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/DEG/",method,"/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/DEG/",method,"/",subornot), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/DEG/",method,"/",subornot,'/deseq2/'), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            #dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/DEG/",method,"/",subornot,'/deseq2), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            write.table(res, file = paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/DEG/",method,"/",subornot,"/deseq2/",empty_list[[i]][1],empty_list[[i]][2],"_",sample_num,".csv"),quote = FALSE, sep = ",", row.names = TRUE,col.names = TRUE)
            ####################################volcano_plot
            print(4)
            res<-list()
            res[[1]]<-results(dds)
            
            res[[1]]$symbol <-full[,1]
            #res[[1]]$ensemble <-featuredata$ensemble
            #res[[1]]
            keyvals.colour <- ifelse(res[[1]]$log2FoldChange < -2&res[[1]]$padj<0.05, 'royalblue',
                                     ifelse(res[[1]]$log2FoldChange > 2&res[[1]]$padj<0.05, 'red',
                                            ifelse(res[[1]]$log2FoldChange< 2 & res[[1]]$log2FoldChange> -2& res[[1]]$padj> 0.05,'darkgreen','grey')))
            names(keyvals.colour)[keyvals.colour == 'red'] <- 'high'
            names(keyvals.colour)[keyvals.colour == 'darkgreen'] <- 'mid'
            names(keyvals.colour)[keyvals.colour == 'royalblue'] <- 'low'
            EnhancedVolcano(toptable = res[[1]], # data-frame of test statistics
                            lab = res[[1]]$symbol, # column name in toptable containing variable names
                            x ='log2FoldChange', # A column name in toptable containing log2 fold change
                            y = 'padj', # A column name in toptable containing p-value
                            pCutoff = 0.05, # Cut-off for statistical significance. (in raw value)
                            FCcutoff = 2,   # Cut-off for absolute log2 fold-change
                            title = method, # Title of the plot
                            subtitle = "Fold Change > 2, padj < 0.05", # Subtitle of the plot
                            ylim = c(0, 15), # Limits of the y-axis
                            pCutoffCol = 'padj',
                            gridlines.minor = F,
                            legendPosition = 'none',
                            selectLab = rownames(res[[1]])[which(names(keyvals.colour) %in% c('high', 'low'))],
                            colCustom = keyvals.colour)
            print(1)
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/Figure/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/Figure/volcano/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/Figure/volcano/",method), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/Figure/volcano/",method,"/",subornot), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/Figure/volcano/",method,'/',subornot,"/deseq2"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            ggsave(filename = paste0('/data/projects/2021_MOPA/SMALL_NMF_V2/result/',data_type,"/",sub_sample_num,'/Figure/volcano/',method,"/",subornot,'/deseq2/',empty_list[[i]][1],empty_list[[i]][2],"_",sample_num,'.pdf'),
                   device = 'pdf',
                   width = 5,
                   height= 5,
                   units = 'in')
            dge <- DGEList(counts =full[,colnames(full)%in%rownames(col_data[col_data$Subtype %in% empty_list[[i]],])], group = col_data[col_data$Subtype %in% empty_list[[i]],][,2])
            dge <- estimateDisp(y = dge)
            et <- exactTest(object = dge)
            top_degs = topTags(object = et, n = "Inf")
        
        
            col_data[col_data$Subtype %in% empty_list[[i]],]
        
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/DEG/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/DEG/",method), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/DEG/",method,"/",subornot), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/DEG/",method,"/",subornot,"/edgeR/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
        
            write.table(as.matrix(top_degs$table), file = paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/DEG/",method,"/",subornot,"/edgeR/",empty_list[[i]][1],empty_list[[i]][2],"_",sample_num,".csv"),quote = FALSE, sep = ",", row.names = TRUE,col.names = TRUE)
        
        
            keyvals.colour <- ifelse(top_degs$table$logFC < -1& top_degs$table$FDR<0.05, 'royalblue',
                                 ifelse(top_degs$table$logFC > 1&top_degs$table$FDR<0.05, 'red',
                                        ifelse(top_degs$table$logFC< 1 & top_degs$table$logFC> -1& top_degs$table$FDR> 0.05,'darkgreen','grey')))
            names(keyvals.colour)[keyvals.colour == 'red'] <- 'high'
            names(keyvals.colour)[keyvals.colour == 'darkgreen'] <- 'mid'
            names(keyvals.colour)[keyvals.colour == 'royalblue'] <- 'low'
            EnhancedVolcano(toptable = top_degs$table, # data-frame of test statistics
                        lab = row.names(top_degs$table), # column name in toptable containing variable names
                        x ='logFC', # A column name in toptable containing log2 fold change
                        y = 'FDR', # A column name in toptable containing p-value
                        pCutoff = 0.05, # Cut-off for statistical significance. (in raw value)
                        FCcutoff = 1,   # Cut-off for absolute log2 fold-change
                        title = method, # Title of the plot
                        subtitle = "Fold Change > 1, padj < 0.05", # Subtitle of the plot
                        ylim = c(0, 15), # Limits of the y-axis
                        pCutoffCol = 'FDR',
                        gridlines.minor = F,
                        legendPosition = 'none',
                        selectLab = rownames(res[[1]])[which(names(keyvals.colour) %in% c('high', 'low'))],
                        colCustom = keyvals.colour)
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/Figure/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/Figure/volcano/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/Figure/volcano/",method), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/Figure/volcano/",method,"/",subornot), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/Figure/volcano/",method,"/",subornot,"/edgeR/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            
            ggsave(filename = paste0('/data/projects/2021_MOPA/SMALL_NMF_V2/result/',data_type,"/",sub_sample_num,'/Figure/volcano/',method,"/",subornot,"/edgeR/",empty_list[[i]][1],empty_list[[i]][2],"_",sample_num,'.pdf'),
               device = 'pdf',
               width = 5,
               height= 5,
               units = 'in')
        }
    }

}, data_types, data_typs)
    

Warning message in DESeqDataSet(se, design = design, ignoreRank):
"some variables in design formula are characters, converting to factors"


[1] 1


using pre-existing size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 978 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] 4
[1] 1


Using classic mode.

Warning message in DESeqDataSet(se, design = design, ignoreRank):
"some variables in design formula are characters, converting to factors"


...

[1] 1


using pre-existing size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 39 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] 4
[1] 1


Using classic mode.



$COAD_paper
NULL

$STAD_paper
NULL

$BRCA_paper
NULL

$LUAD_paper
NULL

$COAD_filter_paper
NULL

$BRCA_filter_paper
NULL

$COVID_B_paper
NULL

$`COVID_CD4 T_paper`
NULL

$`COVID_CD8 T_paper`
NULL

$COVID_Mono_paper
NULL

$COVID_NK_paper
NULL

# PATHWAY

In [30]:
KEGG= msigdbr(species = "human", category = "C2",subcategory = 'CP:KEGG')
REACTOME = msigdbr(species = "human", category = "C2",subcategory = 'CP:REACTOME')
GO = msigdbr(species = "human", category = "C5")
reactome_pathway = split(x = REACTOME$gene_symbol, f = REACTOME$gs_name)
kegg_pathway = split(x = KEGG$gene_symbol, f = KEGG$gs_name)

In [37]:
       full_data <-read.table(paste0('/data/projects/2021_MOPA/SMALL_NMF_V2/data/',data_typ,'/full_data/gene_matrix.csv'),sep=',',header=1)
        sample_label<- read.table(paste0('/data/projects/2021_MOPA/SMALL_NMF_V2/data/',data_typ,'/full_data/subtype.csv'),sep=',',header=1)
        sample_label<- read.table(paste0('/data/projects/2021_MOPA/SMALL_NMF_V2/data/',data_typ,'/full_data/subtype.csv'),sep=',',header=1)
        NMF <- read.table(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/remain_genes/",method,"/",sample_num,".csv"), sep = ",",header=T)
        ####original data DESEQ2
        col_data<-sample_label
        rownames(col_data) <-col_data$X
        NMF <- read.table(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/remain_genes/",method,"/",sample_num,".csv"), sep = ",",header=T)
        if (sub_sample != 'full') #### sub sample
        {
            sub_sample_index <-read.table(paste0('/data1/projects/2021_MOPA/SMALL_NMF_V2/data/',data_typ,'/subsample/',sub_sample_num,'/samples_',sample_num,'.csv'),sep=',',header=1)    
        }
        ####original data DESEQ2
        col_data<-sample_label
        rownames(col_data) <-col_data$X
            
        ######################################################### noisyR removal
        full<-full_data[full_data[,1] %in% NMF[,1],]
        rownames(full)<-full_data[full_data[,1]%in% NMF[,1],][,1]
        full<-full[,-c(1)]
        

Warning message in file(file, "rt"):
"cannot open file '/data1/projects/2021_MOPA/SMALL_NMF_V2/result/COAD_paper/full/remain_genes/NMF/0.csv': No such file or directory"


ERROR: Error in file(file, "rt"): cannot open the connection


In [40]:
methods<-list.files(path=paste0('/data/projects/2021_MOPA/SMALL_NMF_V2/result/',data_type,'/',sub_sample_num,'/remain_genes/'))
    

In [45]:
grepl("COVID", data_types)

[1] FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE

In [50]:
data_types

[1] "COAD_paper"        "STAD_paper"        "BRCA_paper"       
 [4] "LUAD_paper"        "COAD_filter_paper" "BRCA_filter_paper"
 [7] "COVID_B_paper"     "COVID_CD4 T_paper" "COVID_CD8 T_paper"
[10] "COVID_Mono_paper"  "COVID_NK_paper"

In [48]:
degs<-c('deseq2','edgeR')
sample_num<-0
sub_sample <-'full'
sample_num_len <-1
sub='yes'

sub='no'
subornot ='full_sample' ## sub_sample
numnum=0
#print(1)
mapply(function(data_type, data_typ) {
    
    
    if(grepl("COVID", data_type))
    {
        reactome_pathway = split(x = REACTOME$gene_symbol, f = REACTOME$gs_name)
        kegg_pathway = split(x = KEGG$gene_symbol, f = KEGG$gs_name)
        GO_pathway = split(x = GO$gene_symbol, f = GO$gs_name)
    }
    else{
        reactome_pathway = split(x = REACTOME$ensembl_gene, f = REACTOME$gs_name)
        kegg_pathway = split(x = KEGG$ensembl_gene, f = KEGG$gs_name)
        GO_pathway = split(x = GO$ensembl_gene, f = GO$gs_name)
    }
    
    methods<-list.files(path=paste0('/data/projects/2021_MOPA/SMALL_NMF_V2/result/',data_type,'/',sub_sample_num,'/remain_genes/'))
    methods<-c('PCA_norm_5_360_0.05','NMF_norm_5_360_0.05','NMF_norm_10_360_0.001','noisyR','full_genes','PCA_norm_10_360_0.001')
    for(method in methods)
    {
        full_data <-read.table(paste0('/data/projects/2021_MOPA/SMALL_NMF_V2/data/',data_typ,'/full_data/gene_matrix.csv'),sep=',',header=1)
        sample_label<- read.table(paste0('/data/projects/2021_MOPA/SMALL_NMF_V2/data/',data_typ,'/full_data/subtype.csv'),sep=',',header=1)
        sample_label<- read.table(paste0('/data/projects/2021_MOPA/SMALL_NMF_V2/data/',data_typ,'/full_data/subtype.csv'),sep=',',header=1)
        NMF <- read.table(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/remain_genes/",method,"/",sample_num,".csv"), sep = ",",header=T)
        ####original data DESEQ2
        col_data<-sample_label
        rownames(col_data) <-col_data$X
        NMF <- read.table(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/remain_genes/",method,"/",sample_num,".csv"), sep = ",",header=T)
        if (sub_sample != 'full') #### sub sample
        {
            sub_sample_index <-read.table(paste0('/data1/projects/2021_MOPA/SMALL_NMF_V2/data/',data_typ,'/subsample/',sub_sample_num,'/samples_',sample_num,'.csv'),sep=',',header=1)    
        }
        ####original data DESEQ2
        col_data<-sample_label
        rownames(col_data) <-col_data$X
            
        ######################################################### noisyR removal
        full<-full_data[full_data[,1] %in% NMF[,1],]
        rownames(full)<-full_data[full_data[,1]%in% NMF[,1],][,1]
        full<-full[,-c(1)]
        
        
        ## sub sample
        if(sub=='yes')
        {
            full<-full[,sub_sample_index[,2]+1]
            col_data<-col_data[sub_sample_index[,2]+1,]
        }
        
        
        ############### MAKE label combination
        len <-length(unique(sample_label$Subtype))
        #len<-len*(len-1)/2
        empty_list <- vector(mode = "list", length = (len*(len-1))/2)
        unique(sample_label$Subtype)[1]
        num<-1
        for( i in c(1:(length(unique(sample_label$Subtype))-1)))
        {    
            for(j in c((i+1):length(unique(sample_label$Subtype))))
                {
                    empty_list[[num]]<-append(empty_list[[num]],unique(sample_label$Subtype)[i])
                    empty_list[[num]]<-append(empty_list[[num]],unique(sample_label$Subtype)[j])
                    num<-num+1
            }
         }
        len<-len*(len-1)/2
        colnames(full)<-col_data[,1]
        
        
        for( i in c(1:len))
        {
            
            for(deg in degs)
            {
            DEG<-read.csv(file = paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,
                               "/DEG/",method,"/",subornot,"/",deg,"/",empty_list[[i]][1],empty_list[[i]][2],"_",
                               sample_num,".csv"))
            #DEG1<-read.csv(file = paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,
            #       "/DEG/",method,"/",subornot,"/edgeR/",empty_list[[i]][1],empty_list[[i]][2],"_",
            #       sample_num,".csv"))
            if(deg=='deseq2')
            {
                DEG<-DEG[DEG$log2FoldChange> 1 | DEG$log2FoldChange< -1,]
                DEG<-DEG[DEG$padj<0.01,]
                DEG<-na.omit(DEG)
            }
            
            if(deg=='edgeR')
            {
                DEG<-DEG[DEG$logFC> 1 | DEG$logFC< -1,]
                DEG<-DEG[DEG$FDR<0.01,]
                DEG<-na.omit(DEG)
            }
            
            #li2 <- append(li,'r')
            #a<-append(rownames(DEG),rownames(DEG1))
            #DEG<-unique(a)
            DEG<-rownames(DEG)
            
            hyp_obj<- hypeR(DEG, kegg_pathway , test="hypergeometric")
            hyp_obj<-hyp_obj$data[hyp_obj$data$fdr<0.05,]
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot,"/",deg), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot,'/',deg,'/KEGG_v1/'), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            #dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/DEG/",method,"/",subornot,'/deseq2), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            write.table(hyp_obj, file = paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot,'/',deg,"/KEGG_v1/",empty_list[[i]][1],empty_list[[i]][2],"_",sample_num,".csv"),quote = FALSE, sep = "|", row.names = TRUE,col.names = TRUE)
            ####################################volcano_plot

            hyp_obj<- hypeR(DEG, reactome_pathway , test="hypergeometric")
            hyp_obj<-hyp_obj$data[hyp_obj$data$fdr<0.05,]
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot,'/',deg), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot,'/',deg,'/REACTOM_v1/'), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            #dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/DEG/",method,"/",subornot,'/deseq2), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            write.table(hyp_obj, file = paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot,'/',deg,"/REACTOM_v1/",empty_list[[i]][1],empty_list[[i]][2],"_",sample_num,".csv"),quote = FALSE, sep = "|", row.names =TRUE,col.names = TRUE)
            
            
            hyp_obj<- hypeR(DEG, GO_pathway , test="hypergeometric")
            hyp_obj<-hyp_obj$data[hyp_obj$data$fdr<0.05,]
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot,'/',deg), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot,'/',deg,'/GO_v1/'), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            #dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/DEG/",method,"/",subornot,'/deseq2), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            write.table(hyp_obj, file = paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot,'/',deg,"/GO_v1/",empty_list[[i]][1],empty_list[[i]][2],"_",sample_num,".csv"),quote = FALSE, sep = "|", row.names =TRUE,col.names = TRUE)
            
            
            
            DEG<-read.csv(file = paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,
                               "/DEG/",method,"/",subornot,'/',deg,"/",empty_list[[i]][1],empty_list[[i]][2],"_",
                               sample_num,".csv"))
            if(deg=='deseq2')
            {
                DEG<-DEG[DEG$log2FoldChange> 1 | DEG$log2FoldChange< -1,]
                DEG<-DEG[DEG$padj<0.05,]
                DEG<-na.omit(DEG)
            }
            
            if(deg=='edgeR')
            {
                DEG<-DEG[DEG$logFC> 1 | DEG$logFC< -1,]
                DEG<-DEG[DEG$FDR<0.05,]
                DEG<-na.omit(DEG)
            }
            DEG<-rownames(DEG)
            
            
            
            

            hyp_obj<- hypeR(DEG, kegg_pathway , test="hypergeometric")
            hyp_obj<-hyp_obj$data[hyp_obj$data$fdr<0.05,]
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot,'/',deg), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot,'/',deg,'/KEGG_v2/'), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            write.table(hyp_obj, file = paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot,'/',deg,"/KEGG_v2/",empty_list[[i]][1],empty_list[[i]][2],"_",sample_num,".csv"),quote = FALSE, sep = "|", row.names =TRUE,col.names = TRUE)
            ####################################volcano_plot

            hyp_obj<- hypeR(DEG, reactome_pathway , test="hypergeometric")
            hyp_obj<-hyp_obj$data[hyp_obj$data$fdr<0.05,]
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot,'/',deg), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot,'/',deg,'/REACTOM_v2/'), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            write.table(hyp_obj, file = paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot,'/',deg,"/REACTOM_v2/",empty_list[[i]][1],empty_list[[i]][2],"_",sample_num,".csv"),quote = FALSE, sep = "|", row.names = TRUE,col.names = TRUE)
            
            
            hyp_obj<- hypeR(DEG, GO_pathway , test="hypergeometric")
            hyp_obj<-hyp_obj$data[hyp_obj$data$fdr<0.05,]
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/"), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot,'/',deg), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            dir.create(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot,'/',deg,'/GO_v2/'), showWarnings = FALSE, recursive = FALSE, mode = "0777")
            write.table(hyp_obj, file = paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/pathway/",method,"/",subornot,'/',deg,"/GO_v2/",empty_list[[i]][1],empty_list[[i]][2],"_",sample_num,".csv"),quote = FALSE, sep = "|", row.names =TRUE,col.names = TRUE)
            
            
            
        }
    }
}
}, data_types, data_typs)

$COAD_paper
NULL

$STAD_paper
NULL

$BRCA_paper
NULL

$LUAD_paper
NULL

$COAD_filter_paper
NULL

$BRCA_filter_paper
NULL

$COVID_B_paper
NULL

$`COVID_CD4 T_paper`
NULL

$`COVID_CD8 T_paper`
NULL

$COVID_Mono_paper
NULL

$COVID_NK_paper
NULL

In [49]:
print(1)

[1] 1


In [ ]:
 hyp_obj<- hypeR(DEG, kegg_pathway , test="hypergeometric")

In [22]:
            DEG<-read.csv(file = paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,
                               "/DEG/",method,"/",subornot,"/",deg,"/",empty_list[[i]][1],empty_list[[i]][2],"_",
                               sample_num,".csv"))
            #DEG1<-read.csv(file = paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,
            #       "/DEG/",method,"/",subornot,"/edgeR/",empty_list[[i]][1],empty_list[[i]][2],"_",
            #       sample_num,".csv"))
            DEG<-DEG[DEG$log2FoldChange> 1 | DEG$log2FoldChange< -1,]
            DEG<-DEG[DEG$padj<0.01,]
            DEG<-na.omit(DEG)
            
            ##DEG1<-DEG1[DEG1$logFC> 2 | DEG1$logFC< -2,]
            #DEG1<-DEG1[DEG1$FDR<0.01,]
            #DEG1<-na.omit(DEG1)
            
            
            #li2 <- append(li,'r')
            #a<-append(rownames(DEG),rownames(DEG1))
            #DEG<-unique(a)
            DEG<-rownames(DEG)

In [30]:
            DEG<-read.csv(file = paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,
                               "/DEG/",method,"/",subornot,"/",deg,"/",empty_list[[i]][1],empty_list[[i]][2],"_",
                               sample_num,".csv"))

In [31]:
DEG

,logFC,logCPM,PValue,FDR
,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000250641,9.002358,4.3743443,1.033120e-77,1.926045e-73
ENSG00000244355,9.190684,4.5182147,1.081854e-74,1.008451e-70
ENSG00000071575,-2.484389,5.4436352,6.747903e-59,4.193372e-55
ENSG00000112299,-4.347723,4.6831265,4.062101e-53,1.628868e-49
ENSG00000101346,1.912917,8.3672377,4.368577e-53,1.628868e-49
ENSG00000041353,-2.848233,4.6405957,3.877274e-52,1.204734e-48
ENSG00000172315,1.561951,6.1257433,7.553894e-52,2.011818e-48
ENSG00000146054,-3.884081,3.7447050,1.220162e-50,2.843434e-47
ENSG00000124217,1.621053,5.2626169,1.741258e-50,3.606919e-47


In [24]:
hyp_obj<- hypeR(DEG, kegg_pathway , test="hypergeometric")

logFC,logCPM,PValue,FDR
<dbl>,<dbl>,<dbl>,<dbl>


In [42]:
        sample_label<- read.table(paste0('/data/projects/2021_MOPA/SMALL_NMF_V2/data/',data_typ,'/full_data/subtype.csv'),sep=',',header=1)
        NMF <- read.table(paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/remain_genes/",method,"/",sample_num,".csv"), sep = ",")

          

ERROR: Error in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, : line 2 did not have 2 elements


In [40]:
paste0("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/",data_type,"/",sub_sample_num,"/remain_genes/",method,"/",sample_num,".csv")

[1] "/data1/projects/2021_MOPA/SMALL_NMF_V2/result/COAD/full/remain_genes/full_genes_19000/0.csv"

In [47]:
NMF <- read.table("/data1/projects/2021_MOPA/SMALL_NMF_V2/result/COAD/full/remain_genes/full_genes_19000/0.csv")

ERROR: Error in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, : line 2 did not have 2 elements
